In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [60]:
mg = RasterModelGrid((20,30),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
#d_h = mg.add_ones('link', 'hydraulic__diameter')

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)

h = mg.add_zeros('node', 'hydraulic__head')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

#add input to node 310
moulin_Q=1.
mg.at_node['input__discharge'][310]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 0.#400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
Q[mg.active_links] = 1.
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print "Number of links = ", mg.number_of_links
print "Number of nodes = ", mg.number_of_nodes
print "Number of active links = ", mg.number_of_active_links
print "Number of core nodes = ", mg.number_of_core_nodes

#code added for fsolve algorithm
dhdx = mg.add_zeros('link', 'hydraulic_head_gradient', noclobber=False)
net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)


pfn = PresFlowNetwork(mg)
pfn.run_one_step()
mc = MeltCreep(mg, dt=500.)

Number of links =  1150
Number of nodes =  600
Number of active links =  980
Number of core nodes =  504
Number of iterations = 1 tolerance = 1.62351215299
Number of iterations = 2 tolerance = 1.03568144198
Number of iterations = 3 tolerance = 1.04456032978
Number of iterations = 4 tolerance = 1.02135253489
Number of iterations = 5 tolerance = 0.885273612864
Number of iterations = 6 tolerance = 0.575143100974
Number of iterations = 7 tolerance = 0.347789203528
Number of iterations = 8 tolerance = 0.162028666725
Number of iterations = 9 tolerance = 0.0758473672151
Number of iterations = 10 tolerance = 0.0393016535976
Number of iterations = 11 tolerance = 0.0204708976432
Number of iterations = 12 tolerance = 0.00954555149939
Number of iterations = 13 tolerance = 0.00627964606936
Number of iterations = 14 tolerance = 0.0024624942764
Number of iterations = 15 tolerance = 0.00293271841586
Number of iterations = 16 tolerance = 0.00148355923735
Number of iterations = 17 tolerance = 0.00072226

In [21]:
mc.run_one_step()

mean ddh =  1.56115219276


In [61]:
from scipy.optimize import fsolve
def network_residuals(heads):
    h[mg.core_nodes] = heads
    mg.calc_grad_at_link(h, out=dhdx)
    Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    return mg.calc_net_flux_at_node(Q, out=net_node_flux)[mg.core_nodes] + mg.at_node['input__discharge'][mg.core_nodes]
#This function doesn't account for inflow at nodes


In [75]:
mc.run_one_step()
mg.calc_grad_at_link(h, out=dhdx)
mg.at_node['hydraulic__head'][mg.core_nodes] = fsolve(network_residuals, h[mg.core_nodes])
mg.at_link['conduit__discharge'][mg.core_nodes] = np.sign(dhdx[mg.core_nodes])*np.sqrt(np.fabs(dhdx[mg.core_nodes]))

print "mean h =", mean(mg.at_node['hydraulic__head'][mg.core_nodes] )
print "mean Q =", mean(Q)

mean ddh =  -107018.008706
mean h = 34.5467766531
mean Q = 0.0923094001667


NameError: name 'eqh' is not defined

In [76]:
figure()
imshow_grid_at_node(mg, h)

In [11]:
def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='jet', magnitude= False, lw=2):
    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    for i, value in enumerate(values):
        xs = [link_head_x[i],link_tail_x[i]]
        ys = [link_head_y[i],link_tail_y[i]]
        img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
    colorbar(scalarMap)


In [12]:
figure()
plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
print mean(mg.at_link['hydraulic__diameter'])

2.31419872088


In [13]:
figure()
plot_links(mg, 'conduit__discharge', autoscale=True, magnitude=True)
#print mean(mg.at_link['hydraulic__diameter'])

In [16]:
nsteps = 100
step_start = 100
fig1, axs = subplots(2,2,figsize=(9,6))
every = 10
for step in arange(nsteps)+step_start:
    print "step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean()
    #pfn.run_one_step()    
    mg.calc_grad_at_link(h, out=dhdx)
    h = fsolve(network_residuals, h[mg.core_nodes])
    Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(2,2,1)
        imshow_grid_at_node(mg, h)
        subplot(2,2,2)
        plot_links(mg, 'conduit__discharge', vmin=0., vmax=1., magnitude=True, autoscale=False)
        subplot(2,2,3)
        plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
        subplot(2,2,4)
        plot(mg.at_link['hydraulic__diameter'])
        image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
        tight_layout()
        savefig(image_name)
        fig1.clf()

#im_ani = animation.ArtistAnimation(fig1,ims)

step = 100  avg d_h= 2.24879241814
mean ddh =  -0.00755721305848
step = 101  avg d_h= 2.24235235832
mean ddh =  -0.00753408529144
step = 102  avg d_h= 2.23593200738
mean ddh =  -0.00751102818235
step = 103  avg d_h= 2.2295313051
mean ddh =  -0.00748804153114
step = 104  avg d_h= 2.22315019145
mean ddh =  -0.00746512513566
step = 105  avg d_h= 2.21678860655
mean ddh =  -0.00744227879218
step = 106  avg d_h= 2.21044649071
mean ddh =  -0.00741950229585
step = 107  avg d_h= 2.2041237844
mean ddh =  -0.00739679544102
step = 108  avg d_h= 2.19782042829
mean ddh =  -0.00737415802153
step = 109  avg d_h= 2.19153636319
mean ddh =  -0.00735158983087
step = 110  avg d_h= 2.18527153012
mean ddh =  -0.00732909066235
step = 111  avg d_h= 2.17902587025
mean ddh =  -0.00730666030929
step = 112  avg d_h= 2.17279932494
mean ddh =  -0.00728429856502
step = 113  avg d_h= 2.16659183573
mean ddh =  -0.00726200522305
step = 114  avg d_h= 2.16040334432
mean ddh =  -0.00723978007713
step = 115  avg d_h= 2.1542

mean ddh =  -0.00513605022659
step = 227  avg d_h= 1.57026607594
mean ddh =  -0.00512033086819
step = 228  avg d_h= 1.56590266355
mean ddh =  -0.00510465961413
step = 229  avg d_h= 1.56155260579
mean ddh =  -0.00508903631711
step = 230  avg d_h= 1.5572158618
mean ddh =  -0.00507346083028
step = 231  avg d_h= 1.55289239083
mean ddh =  -0.00505793300721
step = 232  avg d_h= 1.54858215227
mean ddh =  -0.00504245270197
step = 233  avg d_h= 1.54428510562
mean ddh =  -0.00502701976902
step = 234  avg d_h= 1.54000121051
mean ddh =  -0.0050116340633
step = 235  avg d_h= 1.5357304267
mean ddh =  -0.00499629544017
step = 236  avg d_h= 1.53147271406
mean ddh =  -0.00498100375545
step = 237  avg d_h= 1.5272280326
mean ddh =  -0.00496575886537
step = 238  avg d_h= 1.52299634244
mean ddh =  -0.00495056062663
step = 239  avg d_h= 1.51877760382
mean ddh =  -0.00493540889634
step = 240  avg d_h= 1.5145717771
mean ddh =  -0.00492030353207
step = 241  avg d_h= 1.51037882279
mean ddh =  -0.00490524439179


mean ddh =  -0.00347976616434
step = 354  avg d_h= 1.11054116225
mean ddh =  -0.00346911345097
step = 355  avg d_h= 1.10758487426
mean ddh =  -0.00345849329105
step = 356  avg d_h= 1.1046376365
mean ddh =  -0.00344790558404
step = 357  avg d_h= 1.10169942131
mean ddh =  -0.00343735022973
step = 358  avg d_h= 1.09877020111
mean ddh =  -0.00342682712816
step = 359  avg d_h= 1.09584994843
mean ddh =  -0.00341633617968
step = 360  avg d_h= 1.09293863586
mean ddh =  -0.00340587728494
step = 361  avg d_h= 1.09003623608
mean ddh =  -0.00339545034486
step = 362  avg d_h= 1.08714272188
mean ddh =  -0.00338505526064
step = 363  avg d_h= 1.08425806609
mean ddh =  -0.00337469193379
step = 364  avg d_h= 1.08138224166
mean ddh =  -0.00336436026606
step = 365  avg d_h= 1.07851522161
mean ddh =  -0.00335406015953
step = 366  avg d_h= 1.07565697904
mean ddh =  -0.00334379151653
step = 367  avg d_h= 1.07280748713
mean ddh =  -0.00333355423967
step = 368  avg d_h= 1.06996671917
mean ddh =  -0.00332334823

mean ddh =  -0.00236377238427
step = 480  avg d_h= 0.801117257695
mean ddh =  -0.00235651183718
step = 481  avg d_h= 0.799109099782
mean ddh =  -0.00234927304372
step = 482  avg d_h= 0.797107110579
mean ddh =  -0.00234205592915
step = 483  avg d_h= 0.795111271613
mean ddh =  -0.00233486041884
step = 484  avg d_h= 0.793121564474
mean ddh =  -0.00232768643823
step = 485  avg d_h= 0.791137970813
mean ddh =  -0.00232053391287
step = 486  avg d_h= 0.789160472349
mean ddh =  -0.00231340276842
step = 487  avg d_h= 0.787189050859
mean ddh =  -0.00230629293061
step = 488  avg d_h= 0.785223688188
mean ddh =  -0.00229920432524
step = 489  avg d_h= 0.783264366241
mean ddh =  -0.00229213687823
step = 490  avg d_h= 0.781311066988
mean ddh =  -0.00228509051556
step = 491  avg d_h= 0.779363772462
mean ddh =  -0.00227806516331
step = 492  avg d_h= 0.777422464757
mean ddh =  -0.00227106074763
step = 493  avg d_h= 0.775487126033
mean ddh =  -0.00226407719473
step = 494  avg d_h= 0.773557738511
mean ddh =

mean ddh =  -0.00160164725743
step = 605  avg d_h= 0.592638501567
mean ddh =  -0.00159654558753
step = 606  avg d_h= 0.591277967066
mean ddh =  -0.00159145681279
step = 607  avg d_h= 0.589921769087
mean ddh =  -0.00158638086997
step = 608  avg d_h= 0.588569896693
mean ddh =  -0.00158131769607
step = 609  avg d_h= 0.587222339004
mean ddh =  -0.00157626722844
step = 610  avg d_h= 0.585879085192
mean ddh =  -0.00157122940469
step = 611  avg d_h= 0.584540124482
mean ddh =  -0.00156620416276
step = 612  avg d_h= 0.583205446152
mean ddh =  -0.00156119144091
step = 613  avg d_h= 0.581875039533
mean ddh =  -0.00155619117771
step = 614  avg d_h= 0.580548894008
mean ddh =  -0.00155120331204
step = 615  avg d_h= 0.579226999011
mean ddh =  -0.00154622778313
step = 616  avg d_h= 0.577909344031
mean ddh =  -0.00154126453053
step = 617  avg d_h= 0.576595918605
mean ddh =  -0.00153631349413
step = 618  avg d_h= 0.575286712323
mean ddh =  -0.00153137461415
step = 619  avg d_h= 0.573981714826
mean ddh =

mean ddh =  -0.00104723403473
step = 730  avg d_h= 0.453154108113
mean ddh =  -0.00104336848884
step = 731  avg d_h= 0.452264976706
mean ddh =  -0.00103951073373
step = 732  avg d_h= 0.451379132776
mean ddh =  -0.00103566075064
step = 733  avg d_h= 0.450496569702
mean ddh =  -0.00103181852117
step = 734  avg d_h= 0.449617280875
mean ddh =  -0.00102798402716
step = 735  avg d_h= 0.448741259704
mean ddh =  -0.0010241572508
step = 736  avg d_h= 0.447868499612
mean ddh =  -0.00102033817455
step = 737  avg d_h= 0.446998994037
mean ddh =  -0.00101652678117
step = 738  avg d_h= 0.446132736432
mean ddh =  -0.0010127230537
step = 739  avg d_h= 0.445269720265
mean ddh =  -0.00100892697549
step = 740  avg d_h= 0.444409939016
mean ddh =  -0.00100513853014
step = 741  avg d_h= 0.443553386182
mean ddh =  -0.00100135770155
step = 742  avg d_h= 0.442700055271
mean ddh =  -0.000997584473897
step = 743  avg d_h= 0.441849939806
mean ddh =  -0.000993818831621
step = 744  avg d_h= 0.441003033323
mean ddh =

mean ddh =  -0.000626652209905
step = 853  avg d_h= 0.366551270957
mean ddh =  -0.000623670152227
step = 854  avg d_h= 0.366019795523
mean ddh =  -0.000620695100544
step = 855  avg d_h= 0.36549085535
mean ddh =  -0.000617727060696
step = 856  avg d_h= 0.364964444464
mean ddh =  -0.000614766038861
step = 857  avg d_h= 0.364440556883
mean ddh =  -0.00061181204156
step = 858  avg d_h= 0.363919186621
mean ddh =  -0.000608865075663
step = 859  avg d_h= 0.363400327687
mean ddh =  -0.000605925148391
step = 860  avg d_h= 0.362883974083
mean ddh =  -0.000602992267321
step = 861  avg d_h= 0.362370119803
mean ddh =  -0.000600066440391
step = 862  avg d_h= 0.361858758836
mean ddh =  -0.000597147675903
step = 863  avg d_h= 0.361349885165
mean ddh =  -0.000594235982524
step = 864  avg d_h= 0.360843492762
mean ddh =  -0.000591331369291
step = 865  avg d_h= 0.360339575595
mean ddh =  -0.000588433845615
step = 866  avg d_h= 0.359838127622
mean ddh =  -0.000585543421277
step = 867  avg d_h= 0.3593391427

mean ddh =  -0.000316700005677
step = 976  avg d_h= 0.318289615358
mean ddh =  -0.000314690612583
step = 977  avg d_h= 0.318021444228
mean ddh =  -0.000312689870042
step = 978  avg d_h= 0.317754978077
mean ddh =  -0.000310697776257
step = 979  avg d_h= 0.317490209538
mean ddh =  -0.000308714329056
step = 980  avg d_h= 0.31722713124
mean ddh =  -0.000306739525897
step = 981  avg d_h= 0.316965735818
mean ddh =  -0.000304773363874
step = 982  avg d_h= 0.316706015908
mean ddh =  -0.000302815839727
step = 983  avg d_h= 0.316447964149
mean ddh =  -0.000300866949844
step = 984  avg d_h= 0.316191573183
mean ddh =  -0.000298926690271
step = 985  avg d_h= 0.315936835655
mean ddh =  -0.000296995056721
step = 986  avg d_h= 0.315683744216
mean ddh =  -0.000295072044576
step = 987  avg d_h= 0.315432291517
mean ddh =  -0.000293157648897
step = 988  avg d_h= 0.315182470216
mean ddh =  -0.000291251864431
step = 989  avg d_h= 0.314934272975
mean ddh =  -0.000289354685618
step = 990  avg d_h= 0.314687692

KeyboardInterrupt: 

In [15]:
mc = MeltCreep(mg, dt=5000.)
#mean(mc.Q)

In [17]:
plot(mg.at_link['hydraulic__diameter'][200:300])


Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902


Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.6482677381

In [23]:
#figure()

pfn.run_one_step()
mc.run_one_step()

Number of iterations = 1 tolerance = 0.00299676026854
Number of iterations = 2 tolerance = 1.34229231785e-05
mean Q= 0.0179517502025


In [41]:
#for m in mc.melt:
#    print m
print max(mc.melt[~np.isnan(mc.melt)])
print max(mc.creep)
#print mc.d_h.mean()
#print mg.at_link['hydraulic__diameter']

3.35041761374e-06
2.12710591355e-06


In [15]:
#mc.run_one_step()

mc._grid.at_link['conduit__discharge']

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])